查询pip下载当前所使用的源

In [1]:
pip config list

global.index-url='https://mirror.sjtu.edu.cn/pypi/web/simple/'
Note: you may need to restart the kernel to use updated packages.


##### 换源

常使用的源：<br />
清华：https://pypi.tuna.tsinghua.edu.cn/simple/ <br />
阿里云：http://mirrors.aliyun.com/pypi/simple/ <br />
中国科技大学：https://pypi.mirrors.ustc.edu.cn/simple/ <br />
华中科技大学：http://pypi.hustunique.com/simple/ <br />
上海交通大学：https://mirror.sjtu.edu.cn/pypi/web/simple/ <br />
豆瓣：http://pypi.douban.com/simple/ <br />

In [ ]:
pip config set global.index-url 源地址

##### 导入包的作用

In [ ]:
# os: 用于与操作系统进行交互，例如文件和文件夹的操作。
# sys: 提供了访问Python解释器的变量和函数的功能。
# sys: 提供了访问Python解释器的变量和函数的功能。
# argparse: 用于解析命令行参数和选项。
import os, sys, glob, argparse
import pandas as pd #提供数据分析工具，主要用于处理和分析数据表格。
import numpy as np #提供多维数组对象和用于处理数组的函数。
from tqdm import tqdm #提供多维数组对象和用于处理数组的函数。

import cv2 #用于图像处理和计算机视觉任务。
from PIL import Image #用于图像处理，类似于OpenCV，但主要用于处理图像。
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# train_test_split:数据集划分 StratifiedKFold:分层交叉验证 KFold:普通的K折交叉验证
import torch
# 初始化随机数生成器的种子
torch.manual_seed(0)
# 这行代码设置PyTorch的CuDNN后端不使用确定性算法。CuDNN是用于GPU加速的深度学习库，通过设置为False，可以在一定程度上提高性能。
torch.backends.cudnn.deterministic = False
# 这行代码启用CuDNN的自动优化机制，以加速深度卷积神经网络的训练。
torch.backends.cudnn.benchmark = True

import torchvision.models as models #提供了许多预训练的深度学习模型，例如VGG等
import torchvision.transforms as transforms #该模块提供了各种用于图像数据预处理和增强的转换函数
import torchvision.datasets as datasets #该模块提供了多个流行的计算机视觉数据集
import torch.nn as nn #构建和操作神经网络的核心模块之一
import torch.nn.functional as F #用于在网络的前向传播过程中执行一些非参数化的操作，激活函数、池化、归一化等
import torch.optim as optim #定义和实现各种优化算法的模块
from torch.autograd import Variable #实现自动微分
from torch.utils.data.dataset import Dataset #数据加载和预处理提供一个统一的接口

import albumentations as A # 用于图像数据增强的强大库
import nibabel as nib #NIFTI 数据格式处理库
import matplotlib.pyplot as plt
from nibabel.viewers import OrthoSlicer3D #用于创建交互式的三维正交切片可视化，用于显示 NIFTI 格式的神经影像数据。

##### 查看nii文件的图像

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# 读取.nii文件
nii_path = r'/home/aistudio/─╘PET═╝╧ё╖╓╬Ў║═╝▓▓б╘д▓т╠Ї╒╜╚№╣л┐к╩¤╛▌/Test/1.nii'
nii_img = nib.load(nii_path)

# 获取图像数据
nii_data = nii_img.get_fdata()

# 可选：查看图像属性
print("图像形状:", nii_data.shape)
print("像素尺寸:", nii_img.header.get_zooms())

# 可根据需要更改切片索引
slice_index = nii_data.shape[2]  // 2
plt.subplot(2,2,1)
plt.imshow(nii_data[:, :, slice_index, 0], cmap='gray') 
plt.subplot(2,2,2)
plt.imshow(nii_data[:, slice_index,:, 0], cmap='gray') 
plt.subplot(2,2,3)
plt.imshow(nii_data[slice_index ,: , : , 0], cmap='gray') 

##### 查看每张nii图片的形状及深度，发现有几种不同的类型

In [ ]:
from collections import defaultdict
print('图片形状：',nib.load(train_path[0]).shape) # 查看一张nii图片，包含高度，宽度，深度和通道数
dict1 = defaultdict(int)  # 创建一个默认值为0的字典，深度
dict2 = defaultdict(int)  # 创建一个默认值为0的字典，宽度和高度
for t in train_path:
  nii_img = nib.load(t)
  dict1[nii_img.shape[-2]] += 1
  dict2[nii_img.shape[1]] += 1
print(dict1)# 每张图片的深度不同，出现频率最高的是深度63，出现了24次
print(dict2) # 每张图片的高度和宽度不同，出现频率最高的是高和宽为128，出现了42次

##### ResNet18的主要结构和一些特点：

- 输入层：ResNet-18接受大小为224x224的彩色图像作为输入。

- 卷积层和池化层：ResNet-18开始以一个7x7的卷积核进行卷积操作，然后经过最大池化层，步长为2。之后，通过四个阶段，每个阶段有不同数量的残差块。每个阶段的特征图尺寸逐渐减小，但特征图的数量逐渐增加。

- 残差块：每个残差块由两个3x3的卷积层组成，卷积层之间有一个跳跃连接，绕过了一个Batch Normalization 层和一个ReLU激活函数。这个跳跃连接可以用一个1x1的卷积层来匹配维度。这种结构允许模型可以学习到残差，避免了训练深度网络时的梯度消失问题。

- 全局平均池化：ResNet-18的最后一层是一个全局平均池化层，将特征图的高和宽维度缩减为1，最终得到每个通道的特征图。

- 全连接层：最后通过一个全连接层将最终的特征映射到类别数。

ResNet-18在许多计算机视觉任务中都表现出色，尤其在图像分类、目标检测和语义分割等领域。它的深度相对较浅，仍然具有强大的表达能力。如果任务需要更深的网络，可以考虑使用更深层次的ResNet变种，如ResNet-34、ResNet-50、ResNet-101和ResNet-152等。

##### 自定义数据集（Dataset）的用法

- 自定义数据集类需要继承自 torch.utils.data.Dataset 类，并实现以下三个方法：__init__、__len__ 和 __getitem__。这样可以方便地在训练时加载、处理和传递数据。

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        """
        初始化方法，接收数据和标签，以及可选的数据变换方法。
        Args:
            data (list or ndarray): 数据列表或数组。
            labels (list or ndarray): 标签列表或数组。
            transform (callable, optional): 数据变换方法。默认为 None。
        """
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        """
        返回数据集的大小（样本数量）。
        """
        return len(self.data)

    def __getitem__(self, index):
        """
        获取数据集中的一个样本数据和其对应的标签。
        Args:
            index (int): 样本索引。
        Returns:
            sample (Tensor or ndarray): 样本数据。
            label (Tensor or int): 样本标签。
        """
        sample = self.data[index]
        label = self.labels[index]

        if self.transform is not None:
            sample = self.transform(sample)  # 对数据应用变换方法

        return sample, label


**一般步骤**
1. 准备数据和标签，可以是列表、数组等形式。
2. 定义数据变换方法（可选），如图像的预处理操作。
3. 创建自定义数据集的实例，并传入数据、标签和变换方法（如果有的话）。
4. 使用数据加载器（torch.utils.data.DataLoader）加载自定义数据集。